## Getting started

We build a simple strategy that goes long when overnight return is negative and exits by the end of the day. If price moves against us, we use a stop loss to get out. We have 15 minute bars of Pepsi (PEP) stock prices

In [1]:
# %%checkall
import pandas as pd
import pyqstrat as pq
import numpy as np

# read 15 minute price bars
pepsi = pd.read_csv('support/pepsi_15_min_prices.csv.gz')[['date', 'c']]
pepsi.columns = ['timestamp', 'c'] 
# the date corresponding to each 15 minute timestamp
pepsi['date'] = pepsi.timestamp.values.astype('M8[D]') 
# compute overnight return
pepsi['overnight_ret'] = np.where(pepsi.date > pepsi.date.shift(1), pepsi.c / pepsi.c.shift(1) - 1, np.nan)
pepsi['overnight_ret_positive'] = (pepsi.overnight_ret > 0)  # whether overnight return is positive
# mark points just before EOD. We enter a marker order at these points so we have one bar to get filled
pepsi['eod'] = np.where(pepsi.date.shift(-2) > pepsi.date, True, False)   
# if the price drops by 0.5% after we enter in the morning take our loss and get out
pepsi['stop_price'] = np.where(np.isfinite(pepsi.overnight_ret), pepsi.c * 0.995, np.nan) 
pepsi['stop_price'] = pepsi.stop_price.fillna(method='ffill')  # fill in the stop price for the rest of the day
pepsi['stop'] = np.where(pepsi.c < pepsi.stop_price, True, False)  # whether we should exit because we are stopped out

strat_builder = pq.StrategyBuilder(data=pepsi)   
strat_builder.add_contract('PEP')
# add the stop price so we can refer to it in
strat_builder.add_indicator('stop_price', pepsi.stop_price.values) 
# convert timestamps from nanoseconds (pandas convention) to minutes so they are easier to view
timestamps = pepsi.timestamp.values.astype('M8[m]')  
prices = pepsi.c.values
# create a dictionary from contract name=>timestamp => price for use in the price function
price_dict = {'PEP': {timestamps[i]: prices[i] for i in range(len(timestamps))}}
# create the price function that the strategy will use for looking up prices 
price_function = pq.PriceFuncDict(price_dict=price_dict)
strat_builder.set_price_function(price_function)

# FiniteRiskEntryRule allows us to enter trades and get out with a limited loss when a stop is hit.
# This enters market orders, if you want to use limit orders, set the limit_increment argument
entry_rule = pq.FiniteRiskEntryRule(
    reason_code='POS_OVERNIGHT_RETURN',  # this is useful to know why we entered a trade
    price_func=price_function, 
    long=True,  # whether we enter a long or short position
    percent_of_equity=0.1,  # set the position size so that if the stop is hit, we lose no more than this
    # stop price is used for position sizing.  Also, we will not enter if the price is already below 
    # stop price for long trades and vice versa
    stop_price_ind='stop_price', 
    single_entry_per_day=True)  # if we are stopped out, do we allow re-entry later in the day

# ClosePositionExitRule fully exits a position using either a market or limit order
# In this case, we want to exit at EOD so we are flat overnight
exit_rule_stop = pq.ClosePositionExitRule(   
    reason_code='STOPPED_OUT',
    price_func=price_function)

# Exit when the stop price is reached
exit_rule_eod = pq.ClosePositionExitRule(
    reason_code='EOD',
    price_func=price_function)

# Setup the rules we setup above so they are only called when the columns below in our data dataframe are true
strat_builder.add_series_rule('overnight_ret_positive', entry_rule, position_filter='zero')
strat_builder.add_series_rule('eod', exit_rule_eod, position_filter='positive')
strat_builder.add_series_rule('stop', exit_rule_stop, position_filter='positive')

# create the strategy and run it
strategy = strat_builder()
strategy.run()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (strategy.py, line 665)

In [ ]:
# Lets evaluate how the strategy did
strategy.df_roundtrip_trades()  

In [ ]:
# Lets evaluate how the strategy did
metrics = strategy.evaluate_returns()

In [ ]:
np.__file__